In [17]:
import pandas as pd
import sqlite3

In [18]:
conn = sqlite3.connect('database.sqlite')      # connect to database

In [19]:
# station ---
sta_sql = "select * from station;"

In [20]:
df_sta = pd.read_sql(sta_sql,conn)             # load dataframe 

In [21]:
df_sta.head(10)

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
5,7,Paseo de San Antonio,37.333798,-121.886943,15,San Jose,8/7/2013
6,8,San Salvador at 1st,37.330165,-121.885831,15,San Jose,8/5/2013
7,9,Japantown,37.348742,-121.894715,15,San Jose,8/5/2013
8,10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013
9,11,MLK Library,37.335885,-121.885660,19,San Jose,8/6/2013


In [22]:
df_sta.shape

(70, 7)

In [29]:
# trip ---

# sqlite> select count(id) from trip;
# 669,959

trip_sql = 'select * from trip;'

df_trip = pd.read_sql(trip_sql,conn)             # load dataframe 

df_trip.head(10)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115
3,4075,1117,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122
4,4076,1118,8/29/2013 9:25,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597
5,4078,3829,8/29/2013 9:31,Redwood City Caltrain Station,22,8/29/2013 10:34,Redwood City Caltrain Station,22,228,Customer,94062
6,4079,995,8/29/2013 9:35,South Van Ness at Market,66,8/29/2013 9:52,South Van Ness at Market,66,327,Subscriber,94102
7,4080,764,8/29/2013 9:36,South Van Ness at Market,66,8/29/2013 9:49,San Francisco Caltrain 2 (330 Townsend),69,315,Subscriber,94117
8,4081,218,8/29/2013 9:38,Mountain View City Hall,27,8/29/2013 9:41,Mountain View City Hall,27,150,Subscriber,97214
9,4084,287,8/29/2013 9:41,Mountain View City Hall,27,8/29/2013 9:46,Mountain View City Hall,27,138,Customer,97214


In [48]:
# busiest starting stations

df_trip.groupby(['start_station_id','start_station_name']).id.count().sort_values(ascending=False).head(20)

start_station_id  start_station_name                           
70                San Francisco Caltrain (Townsend at 4th)         49092
69                San Francisco Caltrain 2 (330 Townsend)          33742
50                Harry Bridges Plaza (Ferry Building)             32934
60                Embarcadero at Sansome                           27713
55                Temporary Transbay Terminal (Howard at Beale)    26089
61                2nd at Townsend                                  25837
74                Steuart at Market                                24838
77                Market at Sansome                                24172
65                Townsend at 7th                                  23724
67                Market at 10th                                   20272
76                Market at 4th                                    20165
64                2nd at South Park                                18496
39                Powell Street BART                        

In [55]:
# busiest starting stations for subscribers only

df_trip[df_trip.subscription_type=='Subscriber'].groupby(['start_station_id','start_station_name']).id.count().sort_values(ascending=False).head(10)

start_station_id  start_station_name                           
70                San Francisco Caltrain (Townsend at 4th)         45842
69                San Francisco Caltrain 2 (330 Townsend)          31962
55                Temporary Transbay Terminal (Howard at Beale)    25050
50                Harry Bridges Plaza (Ferry Building)             23502
61                2nd at Townsend                                  22482
65                Townsend at 7th                                  22118
74                Steuart at Market                                21465
77                Market at Sansome                                21213
67                Market at 10th                                   17548
60                Embarcadero at Sansome                           17268
Name: id, dtype: int64

In [56]:
# busiest starting stations for non subscribers

df_trip[df_trip.subscription_type=='Customer'].groupby(['start_station_id','start_station_name']).id.count().sort_values(ascending=False).head(10)

start_station_id  start_station_name                      
60                Embarcadero at Sansome                      10445
50                Harry Bridges Plaza (Ferry Building)         9432
76                Market at 4th                                4521
39                Powell Street BART                           3932
48                Embarcadero at Vallejo                       3810
71                Powell at Post (Union Square)                3758
74                Steuart at Market                            3373
61                2nd at Townsend                              3355
70                San Francisco Caltrain (Townsend at 4th)     3250
77                Market at Sansome                            2959
Name: id, dtype: int64

In [65]:
### ====================================================================================

In [57]:
# df_trip.groupby(['start_station_name','subscription_type']).id.count().sort_values(ascending=False).head(20)

# df_trip.groupby(['start_station_name','subscription_type']).id.count().unstack()

# df_trip.groupby('start_station_name').id.count().sort_values(ascending=False).head(20).start_station_name

In [45]:
# pivot by duration & subscription type. need to validate 
df_trip.pivot_table('duration',index='start_station_name',columns='subscription_type')

subscription_type,Customer,Subscriber
start_station_name,,
2nd at Folsom,2411.052980,431.706693
2nd at South Park,1994.904578,568.242963
2nd at Townsend,2351.078390,558.952940
5th at Howard,3233.696570,551.576047
Adobe on Almaden,2884.583333,779.144279
Arena Green / SAP Center,6421.527473,530.618375
Beale at Market,4054.634115,515.613843
Broadway St at Battery St,3330.428251,580.384370
Broadway at Main,7027.380952,405.847826


In [24]:
# short trip ---
# skip loading full table for now to save time

# sqlite> select max(start_date) from trip;
# 9/9/2014 9:59
# sqlite> select count(*) from trip where start_date > '9/9/2014 8:00';
# 330

sm_trip_sql = "select * from trip where start_date > '9/9/2014 8:00';"

df_sm_trip = pd.read_sql(sm_trip_sql, conn)

df_sm_trip.head(10)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,443911,444,9/9/2014 8:01,San Francisco Caltrain (Townsend at 4th),70,9/9/2014 8:08,Market at Sansome,77,599,Subscriber,94404
1,443912,1029,9/9/2014 8:01,2nd at South Park,64,9/9/2014 8:18,Market at 4th,76,548,Subscriber,94105
2,443913,280,9/9/2014 8:01,Davis at Jackson,42,9/9/2014 8:06,Commercial at Montgomery,45,537,Subscriber,94111
3,443914,339,9/9/2014 8:01,Steuart at Market,74,9/9/2014 8:07,Embarcadero at Sansome,60,485,Subscriber,94565
4,443915,785,9/9/2014 8:01,San Francisco Caltrain (Townsend at 4th),70,9/9/2014 8:14,Embarcadero at Vallejo,48,343,Subscriber,94065
5,443916,370,9/9/2014 8:01,Temporary Transbay Terminal (Howard at Beale),55,9/9/2014 8:08,2nd at Townsend,61,558,Subscriber,94606
6,443917,721,9/9/2014 8:02,San Francisco Caltrain (Townsend at 4th),70,9/9/2014 8:14,Embarcadero at Vallejo,48,374,Subscriber,94025
7,443920,1002,9/9/2014 8:02,San Francisco Caltrain (Townsend at 4th),70,9/9/2014 8:19,Commercial at Montgomery,45,524,Subscriber,94070
8,443921,534,9/9/2014 8:03,San Francisco Caltrain (Townsend at 4th),70,9/9/2014 8:11,Embarcadero at Folsom,51,636,Subscriber,94070
9,443922,220,9/9/2014 8:03,San Francisco Caltrain 2 (330 Townsend),69,9/9/2014 8:07,Townsend at 7th,65,591,Subscriber,94070


In [25]:
df_sm_trip.shape

(330, 11)

In [28]:
df_sm_trip.groupby('start_station_name').id.count().sort_values(ascending=False).head(10)

start_station_name
San Francisco Caltrain (Townsend at 4th)         45
Harry Bridges Plaza (Ferry Building)             30
Temporary Transbay Terminal (Howard at Beale)    24
2nd at Townsend                                  24
San Francisco Caltrain 2 (330 Townsend)          18
Steuart at Market                                15
Beale at Market                                  13
Market at 10th                                   12
Grant Avenue at Columbus Avenue                  10
Market at 4th                                    10
Name: id, dtype: int64

In [ ]:
# status ---

# sqlite> select count(*) from status;
# 71,984,434

# status_sql = "select * from status;"

# df_status = pd.read_sql(status_sql,conn)             # load dataframe 

# df_status.head(10)

In [ ]:
# df_status.columns

In [ ]:
# weather ---

# sqlite> select count(*) from weather;
# 3665

weather_sql = 'select * from weather;'

df_weather = pd.read_sql(weather_sql,conn)             # load dataframe 

df_weather.columns